In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


# get the observed Flux Ratio
import argparse as ap
from tools import *
import importlib
import sys,os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

parser = ap.ArgumentParser(prog="python {}".format(os.path.basename(__file__)),
                           description="Get FR ",
                           formatter_class=ap.RawTextHelpFormatter)
help_lensname = "name of the lens to process"
help_dataname = "name of the data set to process"
parser.add_argument(dest='lensname', type=str,
                    metavar='lens_name', action='store',
                    help=help_lensname)
parser.add_argument(dest='dataname', type=str,
                    metavar='dataname', action='store',
                    help=help_dataname)
parser.add_argument('-v','--verbose',help="Verbosity",
                    dest="verbose", 
                    default=False,action="store_true")
args = parser.parse_args()
lensname = args.lensname
dataname = args.dataname

present_program(sys.argv[0])
sys.path.append("myconfig/")
config_file = "myconfig_" + lensname + "_" + dataname

config = importlib.import_module(config_file)
analysis_path = pth.Path(config.analysis_directory)
savefig_path = analysis_path/str("FluxRatio")
savefig_path.mkdir()
    
all_FR = []
naming_FR = []
for a, knt in enumerate(config.knotstep_marg): #knotstep of the intr.spline
    saveknt_path = analysis_path/str("analysis_kn"+str(knt))
    for mlt_i, mltype_i in enumerate(config.mltype): #ml type
        if mltype_i=="splml":
            continue
        for mlc_i,ml_config in enumerate(config.ml_config[mlt_i]):
            mllist_name,mlfp = ml_config 
            mlfp_str="_mlfp_"+str(mlfp)
            naming = str("ml"+mltype_i[:-2]+mllist_name+mlfp_str)
            saveml_path = saveknt_path/naming
            with open(str(saveml_path)+"/tot_mag_shift.data","rb") as f:
                mag_shift= pickle.load(f)
            mag_shifts = np.array(mag_shift[1:])- mag_shift[0] #rel to A 
            # mag_shifts are the shift TO BE APPLIED to the lcs in order to superpose them
            # as in lcs_1 + mag_shifts[1] = lcs_2 + mag_shifts[2]
            # THEREFORE the magnitude shift due to the lens is the opposite of that
            # that is mag_lens = - mag_shifts
            # and since F/F' = 10^(-(m-m')/2.5) -> FR = F/F_A(due to lensing) = 10^(-mag_lens/2.5) =\
            # FR = 10^(mag_shifts/2.5) 
            FR = 10**(mag_shifts/2.5) #F_i/F_A
            all_FR.append(FR)
            naming_FR.append(naming)
all_FR = np.transpose(all_FR).tolist()
FR_med = np.median(all_FR,axis=1)

kwargs_FR = {"FR":all_FR,"FR_names":naming_FR,"FR_median":FR_med}
with open(savefig_path/str("FR_res.pkl"),"wb") as f:
    pickle.dump(kwargs_FR,f)


In [ ]:

fg,axes= plt.subplots(1,3,figsize=(15,15)) 
fntsz = 20


colors= config.colors

def fmt_new(y,pos):
    return ""
    
FR_names = ["AB","AC"]

for j,ax in enumerate(axes[:2]):
    ax.yaxis.set_major_formatter(FuncFormatter(fmt_new))
    if j==1:
        ax.set_title("Flux Ratio obtained from lcs analysis",fontsize=fntsz)

    y_rnd = [0.5 + j for j in range(len(all_FR[j])+1,0,-1)]

    for i,fri in enumerate(all_FR[j]):
        colj=colors[int(i%len(colors))]
        ax.scatter(fri ,y_rnd[i], c=colj,label=naming_FR[j])
        #ax.text(fri-.05*len(str(np.round(fri,3))),y_rnd[i]+.2,s=str(np.round(fri,2)),c=colj,fontsize=fntsz)
        ax.text(fri,y_rnd[i],s=str(np.round(fri,2)),c=colj,fontsize=fntsz)
        
    ax.set_xlabel(r"$FR_{"+FR_names[j]+"}$ []",fontsize=fntsz)
    ax.set_ylim(min(y_rnd)-1,max(y_rnd)+1)

    ax.scatter(FR_med[j] ,y_rnd[-1], c="k",label="Median")
    #ax.text(FR_med[j]-.05*len(str(np.round(FR_med[j],3))),y_rnd[-1]+.2,s=str(np.round(FR_med[j],2)),c="k",fontsize=fntsz)
    ax.text(FR_med[j],y_rnd[-1],s=str(np.round(FR_med[j],2)),c="k",fontsize=fntsz)

ax_del = axes[2]
ax_del.axis("off")
ln_lgnd = []
for j in range(len(all_FR[-1])):
    if j >= len(colors):
        colj=colors[int(j%len(colors))]
    else:
        colj=colors[j]
    y_lbl = naming_FR[j] 
    ln, = ax_del.plot(1,1,c=colj,label=y_lbl)
    ln_lgnd.append(ln)


ln, = ax_del.plot(1,1,c="k",label="Median")
ln_lgnd.append(ln)

#if len(ln_lgnd)<20:
ax_del.legend(fontsize=17)   
#else:
#    set_lgnd = ax_del.legend(handles=ln_lgnd[:int(len(ln_lgnd)/2.)],loc="upper left")
#    ax_del.add_artist(set_lgnd)
#    ax_del.legend(handles=ln_lgnd[int(len(ln_lgnd)/2.):],loc="upper center")



plt.tight_layout()
#plt.legend(fontsize=fntsz)
plt.savefig(savefig_path/str("FR_obs.png"))


# In[ ]:


success(sys.argv[0])
